In [1]:
import numpy as np
import pandas as pd
import time
import os
import matplotlib.pyplot as plt
# import dwf

from ctypes import *

%load_ext autoreload
%autoreload 2

In [2]:
from chip_systems import AD2
from chip_systems import ProjectSystem
from chip_systems import Biosensors
from chip_systems import Magnetometers
from chip_systems import InertialSensors

In [ ]:
interface = AD2.AD2_I2C(voltage=3.3, pin=0)

In [ ]:
interface = AD2.AD2_I2C(voltage=0, pin=0)

## IMU

In [ ]:
imu = InertialSensors.ICM20649(interface=interface)

In [5]:
hex(0b1101000)

'0x68'

In [ ]:
# Should be 0xE1 or 225
imu.ReadRegisters('WHO_AM_I')

In [ ]:
SetupIMU(imu)

In [ ]:
ax = imu.ReadAxMgee()
ay = imu.ReadAyMgee()
az = imu.ReadAzMgee()
print('%.2f %.2f %.2f' % (ax, ay, az))

## Magnetometer

In [ ]:
magnetometer = Magnetometers.BMM150(interface=interface)

In [ ]:
magnetometer.WriteRegisters('Power Control Bit', 1)
magnetometer.ReadRegisters(0x4B)

In [ ]:
# Should be 0x32 - Can only be read after chip is turned on
hex(magnetometer.ReadRegisters(0x40)[0])

In [ ]:
# Enter Sleep Mode (0x4C)
magnetometer.WriteRegisters('Opmode', 0b11)

time.sleep(1)
# Enter Normal Mode
magnetometer.WriteRegisters('Opmode', 0b00)

# # Enter Forced Mode
# magnetometer.WriteRegisters('Opmode', 0b01)

In [ ]:
# Set Data Rate (ODR) (0x4C)
# magnetometer.WriteRegisters('Data Rate', 0b100)  # 15 Hz
magnetometer.WriteRegisters('Data Rate', 0b000)  # 10 Hz

In [ ]:
# Set output rate (0x51, 0x52)
magnetometer.WriteRegisters(0x51, 9)  # X/Y Axis
magnetometer.WriteRegisters(0x52, 15)  # Z Axis

In [ ]:
# Turn on All 3 Axes (0x4E)
magnetometer.WriteRegisters('Channel X', 0)  # Enable is 0
magnetometer.WriteRegisters('Channel Y', 0)  # Enable is 0
magnetometer.WriteRegisters('Channel Z', 0)  # Enable is 0

In [ ]:
mxyz = magnetometer.ReadRegisters(0x42, 6)[1]
mx = mxyz[8:16] + mxyz[0:5]
my = mxyz[24:32] + mxyz[16:21]
mz = mxyz[40:48] + mxyz[32:37]

mx = ProjectSystem.TwosComp(int(mx, 2), bits=13)
my = ProjectSystem.TwosComp(int(my, 2), bits=13)
mz = ProjectSystem.TwosComp(int(mz, 2), bits=13)

print(mx, my, mz)

In [ ]:
mxyz = magnetometer.ReadRegisters(0x42, 6)[1]
mx = mxyz[8:16] + mxyz[0:5]
my = mxyz[24:32] + mxyz[16:21]
mz = mxyz[40:48] + mxyz[32:37]

# mx = ProjectSystem.TwosComp(int(mx, 2), bits=13)
# my = ProjectSystem.TwosComp(int(my, 2), bits=13)
# mz = ProjectSystem.TwosComp(int(mz, 2), bits=13)

print(mx, my, mz)

In [ ]:
mxyz

In [ ]:
len(mx)

In [ ]:
int(mx, 2)

In [ ]:
magnetometer.ReadRegisters(0x42)

In [ ]:
magnetometer.ReadRegisters(0x43)

In [ ]:
magnetometer.ReadRegisters(0x4E)

In [ ]:
magnetometer.ReadRegisters(0x52)

In [ ]:
magnetometer.ReadRegisters(0x4C)

In [ ]:
magnetometer.ReadRegisters(0x48)

In [ ]:
magnetometer.ReadRegisters(0x4A)

In [ ]:
magnetometer.ReadRegisters(0x4D)

In [ ]:
# Soft Reset
magnetometer.ReadRegisters(0x4B)
magnetometer.WriteRegisters(0x4B, 0b10000010)
magnetometer.ReadRegisters(0x4B)

In [ ]:
# Perform Self test

# Enter Sleep Mode (0x4C)
magnetometer.WriteRegisters('Opmode', 0b11)

time.sleep(1)

magnetometer.WriteRegisters('Self Test', 1)

In [ ]:
magnetometer.ReadRegisters(0x42)

In [ ]:
magnetometer.ReadRegisters(0x44)

In [ ]:
magnetometer.ReadRegisters(0x46)

In [ ]:
# Trim Registers
BMM150_DIG_X1 = 0x5D
BMM150_DIG_Y1 = 0x5E
BMM150_DIG_Z4_LSB = 0x62
BMM150_DIG_Z4_MSB = 0x63
BMM150_DIG_X2 = 0x64
BMM150_DIG_Y2 = 0x65
BMM150_DIG_Z2_LSB = 0x68
BMM150_DIG_Z2_MSB = 0x69
BMM150_DIG_Z1_LSB = 0x6A
BMM150_DIG_Z1_MSB = 0x6B
BMM150_DIG_XYZ1_LSB = 0x6C
BMM150_DIG_XYZ1_MSB = 0x6D
BMM150_DIG_Z3_LSB = 0x6E
BMM150_DIG_Z3_MSB = 0x6F
BMM150_DIG_XY2 = 0x70
BMM150_DIG_XY1 = 0x71

In [ ]:
magnetometer.ReadRegisters(0x4C)

In [ ]:
magnetometer.ReadRegisters(0x40)

In [ ]:
interface.SetAnalogOut(0, 0)

## Helper Functions

In [ ]:
def SetupIMU(imu):

    imu.WriteRegisters('DEVICE_RESET', 1)
    time.sleep(0.2)

    imu.WriteRegisters('USER_BANK', 0)
    imu.WriteRegisters('SLEEP', 0)
    imu.WriteRegisters('DISABLE_ACCEL', 0)
    imu.WriteRegisters('DISABLE_GYRO', 0)
    time.sleep(0.2)

    SetupGyro(imu)
    SetupAccel(imu)
    SetupTemp(imu)

def SetupAccel(imu):

    imu.WriteRegisters('ACCEL_FS_SEL', 0)
    imu.WriteRegisters('ACCEL_FCHOICE', 1)
    imu.WriteRegisters('ACCEL_DLPFCFG', 0)
    imu.WriteRegisters('ACCEL_SMPLRT_DIV_1', 0)
    imu.WriteRegisters('ACCEL_SMPLRT_DIV_2', 1)
    time.sleep(0.2)

def SetupGyro(imu):

    imu.WriteRegisters('USER_BANK', 2)
    imu.WriteRegisters('GYRO_FS_SEL', 0)
    imu.WriteRegisters('GYRO_FCHOICE', 1)
    imu.WriteRegisters('GYRO_DLPFCFG', 0)
    imu.WriteRegisters('GYRO_SMPLRT_DIV', 1)
    time.sleep(0.2)

def SetupTemp(imu):
    imu.WriteRegisters('TEMP_DLPFCFG', 2)